# 1. Data Exploration

# Imports

In [ ]:
%reload_ext autoreload
%autoreload 2

import data_chaser as dc
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
from data_chaser.plot.plotly import missing_value_heatmap, missing_data_ratios

# Data loading

First we will define the data directory. I recommend `lost-data-chaser/data` such that you can follow along with the notebook. The datasets we will use first are all .csv files from:
- [Meteorite landings](https://catalog.data.gov/dataset/meteorite-landings)
- [Near Earth Comets](https://catalog.data.gov/dataset/near-earth-comets-orbital-elements)
- [Fire and Bolide Reports](https://catalog.data.gov/dataset/fireball-and-bolide-reports)
- [Global Landslide Catalog](https://catalog.data.gov/dataset/global-landslide-catalog)

We will also use an additional time series dataset to try out imputation methods with temporally dependent data.

- [SnowEx17 Time Series Sonic Snow Depth Measurement Array](https://nsidc.org/data/SNEX17_SSD/versions/1)

To access this dataset alongside the others, follow the instructions in the README.md and extract `SnowEx17_snowdepth_15min_V2.csv` to the data directory.

In [ ]:
datadir = os.path.join(os.path.dirname(os.getcwd()), 'data')
fnames = sorted([os.path.join(datadir, fname) for fname in os.listdir(datadir) if fname.endswith('.csv')])
print(fnames)

Now we have the filenames, let's load the data in and inspect the head to get a feeling of the components.

In [ ]:
fire_df = pd.read_csv(fnames[0])
fire_df.head(3)

In [ ]:
landslide_df = pd.read_csv(fnames[1])
landslide_df.head(3)

In [ ]:
meteor_df = pd.read_csv(fnames[1])
meteor_df.head(3)

In [ ]:
comet_df = pd.read_csv(fnames[3])
comet_df.head(3)

Be careful with the snow data, the first 39 rows contain the header, so remove this when reading in the data.

In [ ]:
snow_df = pd.read_csv(fnames[4], skiprows=39)
snow_df['#datetime_MST'] = pd.to_datetime(snow_df['#datetime_MST'])
snow_df = snow_df.set_index('#datetime_MST', drop=True)
snow_df.index = snow_df.index.set_names('timestamp')
snow_df.head(3)

# Visualising the `NaN` distributions

In my opinion, for this challenge we don't *really* mind about the semantic value of each column e.g what does 'P (yr) mean? For building a general case strategy, we onluy care about the type of data (categorical, continuous...) and the sparsity. Therefore we will skimp on any EDA that is **too** in depth into what the data 'means'. Unusual for a data scientist. :) 

## Location of NaNs in each dataset

Before we start implementing a solution, it is important for us to visualise the distribution of missing values (or `NaNs`) for each dataset. This way, we can better understand the sparsity of the data that we're dealing with!

In [ ]:
fig = missing_value_heatmap(fire_df, "fire_df")
fig.show()

In [ ]:
fig = missing_value_heatmap(meteor_df, "meteor_df")
fig.show()

In [ ]:
fig = missing_value_heatmap(landslide_df, "landslide_df")
fig.show()

In [ ]:
fig = missing_value_heatmap(comet_df, "comet_df")
fig.show()

In [ ]:
fig = missing_value_heatmap(snow_df, "snow_df")
fig.show()

From these plots, we can see there are generally 4 types of missing data challenges that we must consider: 
1. Columns with **complete** sparsity (no values)
2. Columns with **high** sparsity (around 90% of values are missing)
3. Columns with **low/medium sparsity** (50% or higher values are present)
4. Completely sparsity (few values in most columns). This type isn't present in these datasets but we can experiment with engineering some.

We must also consider dependencies (or lack of) in the data. Some columns may be measuring samples (rows) with some temporal dependence on each other, e,g a time series from the same signal. Others may be measuring **independent** events. This is important to consider when designing an imputation strategy in which values you can use.

## Visualise NaNs in a time series dataset

Just for fun, let's plot the time series data with gaps in for missing values. :) From this plot, we can see that sensor `SXK5` has many missing values (double click on `SXK5` in the legend to only see its signal!). Have a look at this in comparison to a sensor such as `SXN4`.

In [ ]:
fig = time_series_with_nans(snow_df, 'depth_qc_cm', 'sensor_num')
fig.show()

## Ratio of missing data to present data 

In [ ]:
ratio_fig = missing_data_ratios([comet_df, meteor_df, landslide_df, fire_df, snow_df], ['comet_df', 'meteor_df', 'landslide_df', 'fire_df', 'snow_df'])
ratio_fig.show()

# Summary

Now that we have explored the datasets, we can move on to preprocessing them to prepare them for an imputation strategy in our next notebook. 